In [2]:
import pyscf
import pyscf.qmmm
from pyscf import gto, scf
import numpy as np
import matplotlib.pyplot as plt
from pyscf.geomopt.berny_solver import optimize
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
from pyscf import lib
import inspect
from functools import reduce
from pyscf.scf import cphf
angstrom = 1 / 0.52917721067
from pyscf.scf._response_functions import _gen_rhf_response 
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [3]:
mol = gto.M(atom='H 0 0 0; H 0 0 1.2', unit="Bohr",basis="sto-3g")
mf = scf.RHF(mol)
e=mf.scf()

converged SCF energy = -1.11033388268018


In [4]:
g = mf.Gradients()
g.kernel()

--------------- RHF gradients ---------------
         x                y                z
0 H     0.0000000000     0.0000000000     0.1065855689
1 H     0.0000000000     0.0000000000    -0.1065855689
----------------------------------------------


array([[ 0.        ,  0.        ,  0.10658557],
       [ 0.        ,  0.        , -0.10658557]])

The formula for the gradient is stated in Pople's article (Eq.21) as: 
$$ \frac{\partial E}{\partial x}= \sum_{\mu\nu}P_{\mu\nu}\frac{\partial H_{\mu\nu}}{\partial x}+\frac{1}{2}\sum_{\mu\nu\lambda\sigma}
P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial x}(\mu \lambda | | \nu\sigma)+\frac{\partial V_{nuc}}{\partial x} 
-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial x}
$$
$W$ is an energy weighted density matrix:
$$ W_{\mu\nu}= \sum_i ^{mo.occ.} \epsilon_i c_{\mu i}^\dagger c_{\nu i}
$$

 In order to evaluate the mixed derivative $\frac {\partial^2 E}{\partial Z \partial x } $we need to :<br>
 1)Build the gradient from scratch and understand each individual piece <br>
 2)Derive the expression with respect to $Z$. Will require the $\frac{\partial P}{\partial Z} $ that can be obtained analytically through CPHF once for all normal modes. Might be non trivial $\frac {\partial^2 H}{\partial Z \partial x}$ element. $\partial_Z \partial_x V_{nuc}$ is classical mechanics. $\partial_Z W$ is also easy through the $U$ matrix response of CPHF. $\partial_Z \partial_x(\mu \lambda | | \nu\sigma)=0$ as well as $\partial_Z S=0$

# Let's start inspecting the explicit gradient expression from PySCF

In [8]:
print(inspect.getsource(g.kernel))
print(inspect.getsource(g.grad_elec))

    def kernel(self, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None):
        cput0 = (time.clock(), time.time())
        if mo_energy is None: mo_energy = self.base.mo_energy
        if mo_coeff is None: mo_coeff = self.base.mo_coeff
        if mo_occ is None: mo_occ = self.base.mo_occ
        if atmlst is None:
            atmlst = self.atmlst
        else:
            self.atmlst = atmlst

        if self.verbose >= logger.WARN:
            self.check_sanity()
        if self.verbose >= logger.INFO:
            self.dump_flags()

        de = self.grad_elec(mo_energy, mo_coeff, mo_occ, atmlst)
        self.de = de + self.grad_nuc(atmlst=atmlst)
        if self.mol.symmetry:
            self.de = self.symmetrize(self.de, atmlst)
        logger.timer(self, 'SCF gradients', *cput0)
        self._finalize()
        return self.de

def grad_elec(mf_grad, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None):
    '''
    Electronic part of RHF/RKS gradients

    Args:
        

In [ ]:
"""
This is the important code ----------------------------------------------------------------

hcore_deriv = mf_grad.hcore_generator(mol)                  dH/dx
s1 = mf_grad.get_ovlp(mol)%autocall                         dS/dx   
dm0 = mf.make_rdm1(mo_coeff, mo_occ)                          P

vhf = mf_grad.get_veff(mol, dm0)                           P ( d/dx(ml) || ns )

dme0 = mf_grad.make_rdm1e(mo_energy, mo_coeff, mo_occ)        W

aoslices = mol.aoslice_by_atom()
de = numpy.zeros((len(atmlst),3))
for k, ia in enumerate(atmlst):
    p0, p1 = aoslices [ia,2:]
    h1ao = hcore_deriv(ia) 
    de[k] += numpy.einsum('xij,ij->x', h1ao, dm0)                                    P*dH/dx
# nabla was applied on bra in vhf, *2 for the contributions of nabla|ket>
    de[k] += numpy.einsum('xij,ij->x', vhf[:,p0:p1], dm0[p0:p1]) * 2           P (Pd/dx(ml||ns))    -as constraction on dm0
    de[k] -= numpy.einsum('xij,ij->x', s1[:,p0:p1], dme0[p0:p1]) * 2        W dS/dx

    de[k] += mf_grad.extra_force(ia, locals())   # grid reponce for DFT only

return de
"""

In [16]:
print(inspect.getsource(g.extra_force))

    def extra_force(self, atom_id, envs):
        '''Hook for extra contributions in analytical gradients.

        Contributions like the response of auxiliary basis in density fitting
        method, the grid response in DFT numerical integration can be put in
        this function.
        '''
        return 0



 Think how to get $\frac{\partial^2 H}{\partial Z \partial x}$

In [19]:
print(inspect.getsource(g.hcore_generator))

def hcore_generator(mf, mol=None):
    if mol is None: mol = mf.mol
    with_x2c = getattr(mf.base, 'with_x2c', None)
    if with_x2c:
        hcore_deriv = with_x2c.hcore_deriv_generator(deriv=1)
    else:
        with_ecp = mol.has_ecp()
        if with_ecp:
            ecp_atoms = set(mol._ecpbas[:,gto.ATOM_OF])
        else:
            ecp_atoms = ()
        aoslices = mol.aoslice_by_atom()
        h1 = mf.get_hcore(mol)
        def hcore_deriv(atm_id):
            shl0, shl1, p0, p1 = aoslices[atm_id]
            with mol.with_rinv_at_nucleus(atm_id):
                vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>
                vrinv *= -mol.atom_charge(atm_id)
                if with_ecp and atm_id in ecp_atoms:
                    vrinv += mol.intor('ECPscalar_iprinv', comp=3)
            vrinv[:,p0:p1] += h1[:,p0:p1]
            return vrinv + vrinv.transpose(0,2,1)
    return hcore_deriv

